In [44]:
### IMPORTS AND DRIVER SETUP

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, UnexpectedAlertPresentException, NoAlertPresentException
from selenium.common.exceptions import SessionNotCreatedException, WebDriverException
from selenium.webdriver.chrome.options import Options

import time
import pandas as pd
import os
import re
import sys
import linecache
import numpy as np
import datefinder

from bs4 import BeautifulSoup as soup
from pandas import NA

chrome_driver_binary = "/Users/andrin/Library/CloudStorage/OneDrive-LondonSchoolofEconomics/DS Project/chromedriver"

driver = webdriver.Chrome(chrome_driver_binary)



In [45]:
url = 'https://librarysearch.lse.ac.uk/view/action/uresolver.do?operation=resolveService&package_service_id=10444710030002021&institutionId=2021&customerId=2020&VE=true'

spec_url = "https://global-factiva-com.gate3.library.lse.ac.uk/ha/default.aspx?page_driver=searchBuilder_Search#./!?&_suid=1699211604258013312993362481285"

In [46]:
driver.get(url)

In [51]:
inspect_urls3 = []

while True:
    try:
        elems = driver.find_elements_by_xpath("//a[@href]")
        for elem in elems:
            inspect_urls3.append(elem.get_attribute("href"))

        time.sleep(5)

        driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.nextItem"))))
        driver.find_element(By.CSS_SELECTOR, 'a.nextItem').click()
        print("Navigating to Next Page")

        time.sleep(5)

    except UnexpectedAlertPresentException as uap:
        try:
            driver.switch_to.alert.accept()
            print(uap)
            driver.refresh()
        except NoAlertPresentException as nap:
            print(nap)
            driver.refresh()

        
    except (TimeoutException, WebDriverException) as e:
        time.sleep(2)
        print("Last page reached")
        break

display(inspect_urls3)

Navigating to Next Page
Navigating to Next Page
Navigating to Next Page
Navigating to Next Page
Navigating to Next Page
Navigating to Next Page
Last page reached


['https://global-factiva-com.gate3.library.lse.ac.uk/ha/default.aspx?page_driver=searchBuilder_Search#PageBaseForm',
 'https://global-factiva-com.gate3.library.lse.ac.uk/sb/SimpleSearch.aspx?NAPC=p',
 'https://global-factiva-com.gate3.library.lse.ac.uk/sb/SimpleSearch.aspx?NAPC=p',
 'https://global-factiva-com.gate3.library.lse.ac.uk/sb/default.aspx?NAPC=S',
 'https://global-factiva-com.gate3.library.lse.ac.uk/sb/default.aspx?NAPC=S',
 'https://global-factiva-com.gate3.library.lse.ac.uk/mss/default.aspx?NAPC=G&frS=results',
 'https://global-factiva-com.gate3.library.lse.ac.uk/ha/default.aspx?page_driver=searchBuilder_Search#',
 'https://global-factiva-com.gate3.library.lse.ac.uk/ht/default.aspx?NAPC=T',
 'https://global-factiva-com.gate3.library.lse.ac.uk/ht/default.aspx?NAPC=T',
 'https://global-factiva-com.gate3.library.lse.ac.uk/mt/default.aspx?NAPC=T',
 'https://viewer-factiva-com.gate3.library.lse.ac.uk/newsletters/list?napc=2&sa_from=GL',
 'https://viewer-factiva-com.gate3.librar

In [63]:
all_article_links = []

for link in inspect_urls:
    if "accessionno" in link:
        all_article_links.append(link)


short_links = []

for link in all_article_links:
    short_link = re.sub("&fcpil=en&napc=S&sa_from=&cat=a&page_driver=searchBuilder_Search", '', link)   
    short_link = re.sub("https://global-factiva-com.gate3.library.lse.ac.uk/du/article.aspx/?accessionno=BIZINS0020231103ejb3001bb&drn=drn:archive.newsarticle.", '', link)
    short_links.append(short_link)


unique_indices = sorted(np.unique(short_links, return_index=True)[1].tolist())

unique_articles = []

for i in unique_indices:
    unique_articles.append(all_article_links[i])


display(unique_articles)
print(len(unique_articles))

['https://global-factiva-com.gate3.library.lse.ac.uk/du/article.aspx/?accessionno=WSJO000020231031ejav004s9&drn=drn:archive.newsarticle.WSJO000020231031ejav004s9&fcpil=en&napc=S&sa_from=&cat=a&page_driver=searchBuilder_Search',
 'https://global-factiva-com.gate3.library.lse.ac.uk/du/article.aspx/?accessionno=WSJO000020231031ejav000xd&drn=drn:archive.newsarticle.WSJO000020231031ejav000xd&fcpil=en&napc=S&sa_from=&cat=a&page_driver=searchBuilder_Search',
 'https://global-factiva-com.gate3.library.lse.ac.uk/du/article.aspx/?accessionno=WSJPOD0020231031ejav000b5&drn=drn:archive.newsarticle.WSJPOD0020231031ejav000b5&fcpil=en&napc=S&sa_from=&cat=a&page_driver=searchBuilder_Search',
 'https://global-factiva-com.gate3.library.lse.ac.uk/du/article.aspx/?accessionno=J000000020231031ejav0001z&drn=drn:archive.newsarticle.J000000020231031ejav0001z&fcpil=en&napc=S&sa_from=&cat=a&page_driver=searchBuilder_Search',
 'https://global-factiva-com.gate3.library.lse.ac.uk/du/article.aspx/?accessionno=WSJO00

3351


In [66]:
### SCRAPING ARTICLES FUNCTION

def Scrape(article_urls,scraped_articles=None):
    
    if scraped_articles is None:
        scraped_articles = []


    error_articles = []

    for article_number, article in enumerate(article_urls):

        driver.get(article)

        time.sleep(3)

        #Grab All Text Content
        all_text = str(driver.find_element_by_xpath("/html/body").text)

        #Find Headline
        try:
            headline = str(driver.find_element_by_xpath("//*[@class='enHeadline']").text).replace("FT.com site : ","")
        except NoSuchElementException:
            headline = pd.NA
            print(article_number+1," NO HEADLINE DETECTED - CONTINUING...")
            

        #Find Author
        try:
            author = driver.find_element_by_xpath("//*[@class='author']").text
        except NoSuchElementException:
            author = pd.NA
            print(article_number+1," NO AUTHOR DETECTED - CONTINUING...")
        
        #Find Text Paragraphs
        try:
            text_lines = driver.find_elements_by_xpath("//*[@class='articleParagraph enarticleParagraph']")
        except NoSuchElementException:
            text_lines = pd.NA
            print(article_number+1," !!! WARNING: NO TEXT CONTENT DETECTED - CONTINUING...")

        
        #Find Date
        dates_in_text = list(datefinder.find_dates(all_text, strict = True))
        if len(dates_in_text) > 0:
            date = str(dates_in_text[0])[:10]
        else:
            date = pd.NA

        #Aggregate Article Text Contents 
        content = ""
        for line in text_lines:
            content = content + str(line.text)+"\n"

        #Article Did Not Load Warning
        if "We are unable to process your request at this time. Please try again in a few minutes." in str(all_text):
            print("ERROR: ARTICLE DID NOT LOAD !!!")
            time.sleep(3)
            driver.refresh()
            time.sleep(3)
            error_articles.append(article)
            error_message = "ARTICLE DID NOT LOAD"
        else:
            error_message = pd.NA

        #Create Article Element and Add to List of Scraped Articles
        article_element = [article_number+1,headline,author,date,content,error_message]
        #print(article_element)
        
        scraped_articles.append(article_element)


        time.sleep(1)

        print("Article Number: ", article_number+1, " Progress: ", round((article_number+1)*100/len(article_urls),2), "%")

    if len(error_articles) > 0:
        Scrape(error_articles,scraped_articles)

    print(len(scraped_articles))
            
    return scraped_articles


In [67]:
article_list_scraped = Scrape(unique_articles)
articles_df = pd.DataFrame(article_list_scraped)

articles_df.columns = ['ARTICLE_NUMBER','TITLE','AUTHOR', 'DATE','TEXT','ERRORS']


articles_df.to_csv("Factiva_WSJ.csv", encoding='utf-8', index=False)

Article Number:  1  Progress:  0.03 %
Article Number:  2  Progress:  0.06 %
3  NO AUTHOR DETECTED - CONTINUING...
Article Number:  3  Progress:  0.09 %
Article Number:  4  Progress:  0.12 %
Article Number:  5  Progress:  0.15 %
Article Number:  6  Progress:  0.18 %
Article Number:  7  Progress:  0.21 %
Article Number:  8  Progress:  0.24 %
Article Number:  9  Progress:  0.27 %
Article Number:  10  Progress:  0.3 %
Article Number:  11  Progress:  0.33 %
Article Number:  12  Progress:  0.36 %
Article Number:  13  Progress:  0.39 %
14  NO AUTHOR DETECTED - CONTINUING...
Article Number:  14  Progress:  0.42 %
Article Number:  15  Progress:  0.45 %
Article Number:  16  Progress:  0.48 %
Article Number:  17  Progress:  0.51 %
Article Number:  18  Progress:  0.54 %
19  NO AUTHOR DETECTED - CONTINUING...
Article Number:  19  Progress:  0.57 %
20  NO AUTHOR DETECTED - CONTINUING...
Article Number:  20  Progress:  0.6 %
Article Number:  21  Progress:  0.63 %
Article Number:  22  Progress:  0.66 

In [65]:
import csv

file_path = 'factiva_wsj_urls.csv'

# Writing to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(unique_articles)

In [2]:
import requests as res

obs = res.get("https://advance.lexis.com/search/?pdmfid=1519360&crid=3d54e755-5163-42c9-847b-038525de41a6&pdsearchtype=SearchBox&pdtypeofsearch=searchboxclick&pdstartin=urn%3Ahlct%3A16&pdsearchterms=Elon+Musk&pdtimeline=01%2F01%2F2022+to+31%2F10%2F2023%7Cbetween%7CDD%2FMM%2FYYYY&pdpsf=&pdquerytemplateid=&pdsf=&ecomp=qdbhkkk&prid=76818b77-2020-4fd0-a12d-aacb53058c4b")

In [3]:
obs.content

b'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\n<!doctype html>\n\t<html lang="en">\n\t\t<head>\n\n\n\n<!doctype html>\n\t<html lang="en">\n\t\t<head>\n\t\t\n\t\t\t<!-- OneTrust Cookies Consent Notice start for signin.lexisnexis.com -->\n\t\t\t<script type="text/javascript" src=https://cdn.cookielaw.org/consent/fb54e94d-fcb0-4620-a945-31cd0b28711f/OtAutoBlock.js></script>\n\t\t\t<script src=https://cdn.cookielaw.org/scripttemplates/otSDKStub.js  type="text/javascript" charset="UTF-8" data-domain-script="fb54e94d-fcb0-4620-a945-31cd0b28711f" ></script>\n\t\t\t<script type="text/javascript">\n\t\t\t\tfunction OptanonWrapper() { }\n\t\t\t</script>\n\t\t\t<!-- OneTrust Cookies Consent Notice end for signin.lexisnexis.com -->\n\t\t\n\t\t\t\n\n\t\t\t<meta charset="utf-8" />\n\t\t\t<meta http-equiv="X-UA-Compatible" content="IE=10" >\n\t\t\t<meta name="viewport" content="initial-scale=1">\n\t\t\t<style>\n\n\t\t\t</style>\n\n\t\t\t\n\t\t\t\t\

In [4]:
obs = res.get("https://advance.lexis.com/search/?pdmfid=1519360&crid=3d54e755-5163-42c9-847b-038525de41a6&pdsearchtype=SearchBox&pdtypeofsearch=searchboxclick&pdstartin=urn%3Ahlct%3A16&pdsearchterms=Jeff+Bezos&pdtimeline=01%2F01%2F2022+to+31%2F10%2F2023%7Cbetween%7CDD%2FMM%2FYYYY&pdpsf=&pdquerytemplateid=&pdsf=&ecomp=qdbhkkk&prid=76818b77-2020-4fd0-a12d-aacb53058c4b")

In [5]:
obs.content

b'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\n<!doctype html>\n\t<html lang="en">\n\t\t<head>\n\n\n\n<!doctype html>\n\t<html lang="en">\n\t\t<head>\n\t\t\n\t\t\t<!-- OneTrust Cookies Consent Notice start for signin.lexisnexis.com -->\n\t\t\t<script type="text/javascript" src=https://cdn.cookielaw.org/consent/fb54e94d-fcb0-4620-a945-31cd0b28711f/OtAutoBlock.js></script>\n\t\t\t<script src=https://cdn.cookielaw.org/scripttemplates/otSDKStub.js  type="text/javascript" charset="UTF-8" data-domain-script="fb54e94d-fcb0-4620-a945-31cd0b28711f" ></script>\n\t\t\t<script type="text/javascript">\n\t\t\t\tfunction OptanonWrapper() { }\n\t\t\t</script>\n\t\t\t<!-- OneTrust Cookies Consent Notice end for signin.lexisnexis.com -->\n\t\t\n\t\t\t\n\n\t\t\t<meta charset="utf-8" />\n\t\t\t<meta http-equiv="X-UA-Compatible" content="IE=10" >\n\t\t\t<meta name="viewport" content="initial-scale=1">\n\t\t\t<style>\n\n\t\t\t</style>\n\n\t\t\t\n\t\t\t\t\